In [1]:
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import pandas as pd
import seaborn as sns
import numpy as np
from scipy.stats.mstats import winsorize
from sklearn.impute import SimpleImputer
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV

In [2]:
# Dataset available at https://data.cityofnewyork.us/City-Government/Good-Food-Purchasing-Data/usrf-za7k
path = 'Good_Food_Purchasing_Data.csv'
df_0 = pd.read_csv(path)

In [3]:
df_0.shape

(17208, 12)

In [4]:
df_0.columns

Index(['Agency', 'Time Period', 'Food Product Group', 'Food Product Category',
       'Product Name', 'Product Type', 'Origin Detail', 'Distributor',
       'Vendor', '# of Units', 'Total Weight in lbs', 'Total Cost'],
      dtype='object')

In [5]:
df_0.head()

,Agency,Time Period,Food Product Group,Food Product Category,Product Name,Product Type,Origin Detail,Distributor,Vendor,# of Units,Total Weight in lbs,Total Cost
0,Administration for Childrens Services,2018-2019,Meals,Meals,"baby food, dinner, assorted","Stage 2 Baby Food Dinner, Assorted Variety",Beech-Nut,Babylab Inc.,Babylab Inc.,40.0,100.0,560.0
1,Administration for Childrens Services,2018-2019,Meals,Meals,"baby food, dinner, assorted","Stage 3 Baby Food Dinner, Assorted Variety",Beech-Nut,Babylab Inc.,Babylab Inc.,40.0,106.0,600.0
2,Administration for Childrens Services,2018-2019,Meals,Meals,"baby food, formula","Premium Nursette Ready-to-Feed Infant Formula,...",Enfamil,Babylab Inc.,Babylab Inc.,200.0,1200.0,16200.0
3,Administration for Childrens Services,2018-2019,Beverages,Beverages,"juice, fruit punch, ss","100% Juice Variety Pack, 4 fl oz bottles",Enfamil,Babylab Inc.,Babylab Inc.,100.0,600.0,1940.0
4,Administration for Childrens Services,2018-2019,Condiments & Snacks,Condiments & Snacks,"cereal, baby, oatmeal",Single Grain Oatmeal Baby Cereal,Enfamil,Babylab Inc.,Babylab Inc.,10.0,60.0,290.0


In [6]:
df_0.dtypes

Agency                    object
Time Period               object
Food Product Group        object
Food Product Category     object
Product Name              object
Product Type              object
Origin Detail             object
Distributor               object
Vendor                    object
# of Units               float64
Total Weight in lbs      float64
Total Cost               float64
dtype: object

In [7]:
df_0.describe()

,# of Units,Total Weight in lbs,Total Cost
count,1.715200e+04,1.715000e+04,3.767000e+03
mean,1.915848e+04,2.886157e+04,1.398539e+05
std,5.987577e+05,3.260171e+05,4.243162e+05
min,-3.600000e+01,-2.330000e+02,0.000000e+00
25%,3.000000e+00,2.200000e+01,1.210000e+03
50%,1.700000e+01,1.470000e+02,1.570800e+04
75%,2.110000e+02,1.811500e+03,9.308300e+04
max,5.050947e+07,2.525474e+07,9.110741e+06


## Cleaning
From the cell above, I noticed a couple of things:
- min of `Total Weight in lbs` and `# of Units` is negative, which doesn't make sense.
- min `Total Cost` is 0 which would make sense if either the item being sold was sold for free or `# of Units` for it was 0. Both of those features being 0 when `Total Cost` is 0 doesn't make sense.

In [8]:
df_0.isna().sum() # check for null values per column

Agency                       0
Time Period                  0
Food Product Group           0
Food Product Category        0
Product Name                 1
Product Type                 0
Origin Detail             2149
Distributor                911
Vendor                     486
# of Units                  56
Total Weight in lbs         58
Total Cost               13441
dtype: int64

In [ ]:
# Dropping rows with NaN values in any of the specified columns
# df = df.dropna(subset=['# of Units', 'Total Weight in lbs', 'Total Cost'])
# df[(df["Total Cost"].isna()) | (df["# of Units"].isna()) | (df["Total Weight in lbs"].isna())]

In [ ]:
# Set Total Cost and Total Weight in lbs to 0 where # of Units = 0
# df.loc[df["# of Units"] == 0, ["Total Cost"]] = 0
# df

### Rows where the `Total Weight in lbs` or `# of Units` or `Total Cost` is less than 0

In [9]:
df_0[(df_0["Total Weight in lbs"] < 0) | (df_0["# of Units"] < 0) | (df_0["Total Cost"] < 0)].shape

(6, 12)

In [10]:
df_0.shape

(17208, 12)

### Drop the rows you found above

In [11]:
df_1 = df_0[~((df_0["Total Weight in lbs"] < 0) | (df_0["# of Units"] < 0) | (df_0["Total Cost"] < 0))]
df_1.shape

(17202, 12)

#### Rows where `# of Units` or `Total Weight in lbs` is 0 but `Total Cost` is not 0

In [12]:
df_1[((df_1['# of Units'] == 0) | (df_1['Total Weight in lbs'] == 0)) & (df_1["Total Cost"] != 0)].shape

(960, 12)

### Drop the rows you found above

In [13]:
df_2 = df_1[~(((df_1['# of Units'] == 0) | (df_1['Total Weight in lbs'] == 0)) & (df_1["Total Cost"] != 0))]

#### Rows where `# of Units` and `Total Weight in lbs` is 0 but `Total Cost` is not 0

In [14]:
df_2[((df_2['# of Units'] == 0) & (df_2['Total Weight in lbs'] == 0)) & (df_2["Total Cost"] != 0)].shape

(0, 12)

In [15]:
df_2.describe()

,# of Units,Total Weight in lbs,Total Cost
count,1.619100e+04,1.618800e+04,3.442000e+03
mean,1.985198e+04,3.057655e+04,1.341548e+05
std,6.162235e+05,3.354868e+05,4.223606e+05
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.000000e+00,3.000000e+01,1.144750e+03
50%,1.700000e+01,1.865000e+02,1.436600e+04
75%,2.005000e+02,2.176000e+03,8.852700e+04
max,5.050947e+07,2.525474e+07,9.110741e+06


In [16]:
df_2.isna().sum()

Agency                       0
Time Period                  0
Food Product Group           0
Food Product Category        0
Product Name                 1
Product Type                 0
Origin Detail             1842
Distributor                759
Vendor                     338
# of Units                  51
Total Weight in lbs         54
Total Cost               12800
dtype: int64

In [17]:
df_2.shape

(16242, 12)

## Impute if you need to

In [18]:
# check what percentage of target values are missing
def calculate_percentage(part, whole):
    part = np.array(part, dtype=float)
    whole = np.array(whole, dtype=float)
    
    # Handle division by zero
    if np.any(whole == 0):
        raise ValueError("Whole cannot be zero")
    
    return (part / whole) * 100

# Example usage with scalars
print(calculate_percentage(df_2["Total Cost"].isnull().sum(), df_2.shape[0]))

78.80802856791036


In [19]:
df_2.dropna(subset=['Total Cost'], inplace=True)

C:\Users\rahul\AppData\Local\Temp\ipykernel_14364\2399444282.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2.dropna(subset=['Total Cost'], inplace=True)


In [20]:
df_2.isna().sum()

Agency                     0
Time Period                0
Food Product Group         0
Food Product Category      0
Product Name               1
Product Type               0
Origin Detail            124
Distributor              759
Vendor                   338
# of Units                 1
Total Weight in lbs       14
Total Cost                 0
dtype: int64

## Boxplots

In [ ]:
sns.boxplot(x=df_2["Total Cost"])

In [ ]:
sns.boxplot(x=df_2["Total Weight in lbs"])

In [ ]:
sns.boxplot(x=df_2["# of Units"])

## EDA

## What is the distribution of Total Cost of the foods?

In [ ]:
sns.histplot(data=df_2, x="Total Cost")

## Any strong correlations between features?

In [ ]:
filtered_df = df_2.iloc[:, 9:12]
# correlation matrix
myColors = ("red","white","blue")
cmap1 = clr.LinearSegmentedColormap.from_list('Custom', myColors, len(myColors))
corr = filtered_df.corr()
mask = np.zeros_like(corr, dtype=bool)
#mask[np.triu_indices_from(mask)]= True
plt.figure(figsize=(20,10))
sns.heatmap(corr, mask = mask, vmin = -1, vmax = 1, annot=True)

In [ ]:
# Let Target be Total_cost
x_vars = ["# of Units", "Total Weight in lbs"]
y_vars = ["Total Cost"]
sns.pairplot(filtered_df, x_vars=x_vars, y_vars=y_vars)

## How many foods per Food Product Group?

In [ ]:
sns.countplot(df_2, y="Food Product Group", orient='h')

## How many foods per food product category?

In [ ]:
sns.countplot(df_2, y="Food Product Category", orient='h')

## More Boxplots 

In [ ]:
plt.figure(figsize=(10, 20))  # Adjust the figure size as needed
sns.boxplot( x="Total Cost", y="Food Product Category", data=df_2)

In [ ]:
plt.figure(figsize=(10, 20))  # Adjust the figure size as needed
sns.boxplot( x="Total Cost", y="Food Product Group", data=df_2)

In [ ]:
plt.figure(figsize=(10, 20))  # Adjust the figure size as needed
sns.boxplot( x="Total Cost", y="Vendor", data=df_2)

In [ ]:
plt.figure(figsize=(10, 20))  # Adjust the figure size as needed
sns.boxplot( x="Total Cost", y="Distributor", data=df_2)
plt.tight_layout()  # Adjust layout for better visibility

## Show top 5 highest priced products from top 5 vendors

In [ ]:
df_2["Distributor"].value_counts() \
.head(10) \
.plot(kind="bar", title="Top Distributors")

In [ ]:
# winsorize Total Cost column to get rid of outliers
# df.loc[:, "Total Cost"] = winsorize(df["Total Cost"], limits=(None, 0.0005))

In [ ]:
# df_2.query('Total Cost > 1000')["Distributor"].value_counts() \
# .head(10) \
# .plot(kind="bar", title="Top Distributors whose food products were priced at greater than 1000 USD")

## Scaling & Encoding
- Scale: '# of Units', 'Total Weight in lbs', 'Total Cost'
- Encode: 'Food Product Category', 'Food Product Group'

# review all encoding techniques and normalizing techniques

In [21]:
df_2.describe()

,# of Units,Total Weight in lbs,Total Cost
count,3.441000e+03,3.428000e+03,3.442000e+03
mean,8.749927e+04,1.235444e+05,1.341548e+05
std,1.332750e+06,7.083248e+05,4.223606e+05
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+02,7.237500e+02,1.144750e+03
50%,8.790000e+02,9.169000e+03,1.436600e+04
75%,4.968000e+03,6.730325e+04,8.852700e+04
max,5.050947e+07,2.525474e+07,9.110741e+06


In [22]:
# filter out some columns
df = df_2[[# 'Agency', 'Time Period', 
    #    'Product Name', 'Product Type', 'Origin Detail', 'Distributor',
    #    'Vendor', 
    #    'Food Product Category',
         'Vendor',  'Food Product Group', '# of Units', 'Total Weight in lbs', 'Total Cost']].copy()

In [23]:
# Define numeric and categorical features
numeric_features = ['# of Units', 'Total Weight in lbs']
categorical_features = ['Vendor', 'Food Product Group']

# Create transformers
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Create ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

xgb_model = XGBRegressor()

model = Pipeline(steps=[("preprocessor", preprocessor), ("model", xgb_model)])

#### exhaustively search for the optimal hyperparameters

In [24]:
# Split data into features and target
X = df.drop('Total Cost', axis=1)
y = df['Total Cost']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
# set up our search grid
param_grid = {"model__max_depth":    [4, 5, 6],
              "model__n_estimators": [500, 600, 700],
              "model__learning_rate": [0.01, 0.015]}

# try out every combination of the above values
search = GridSearchCV(model, param_grid, cv=5).fit(X_train, y_train)

print("The best hyperparameters are ",search.best_params_)
print("The best score is ",search.best_score_)

The best hyperparameters are  {'model__learning_rate': 0.015, 'model__max_depth': 4, 'model__n_estimators': 700}
The best score is  0.7915024092434398


## Evaluate model


In [31]:
# Calculate MAE
y_pred = search.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print('Mean Absolute Error:', mae)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

# Calculate RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Mean Absolute Error: 57229.32041287249
Mean Squared Error: 81972006786.9568
Root Mean Squared Error: 286307.53882312775


c:\Users\rahul\OneDrive\Documents\Desktop\food_cost_prediction\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [33]:
pd.DataFrame({'Actual': y_test[5:10], 'Prediction': search.predict(X_test)[5:10]})

,Actual,Prediction
16837,144500.0,95716.453125
15411,1663.0,99.245338
15272,453905.0,432688.625000
16767,189.0,12219.028320
15510,6927.0,7606.429688
